# Products Recomendation Example

In [124]:
%%capture
%load_ext autoreload
%autoreload 2

In [125]:
from src.repository import DatabaseClient, to_table, \
    CityRepository, CountryRepository, RegionRepository, UserRepository, \
    City, Country, Region, User

from src.entity_loader import EntityLoader, \
    CityMapper, RegionMapper, UserMapper, CountryMapper, \
    CountryFilter, CityFilter

## Prepare data access

####  1. Create database client

In [126]:
USERNAME = PASSWORD = "neo4j"
URL = "bolt://localhost:7687"

client = DatabaseClient(URL, USERNAME, PASSWORD)

### 2. Create repositories

In [127]:
cityRepository = CityRepository(client)

countryRepository = CountryRepository(client)

regionRepository = RegionRepository(client)

userRepository = UserRepository(client)

## Prepare database

#### 1. Remove all nodes

In [128]:
# client.run("MATCH (n:City) DETACH DELETE n")
client.run("MATCH (n:User) DETACH DELETE n")
client.run("MATCH (n:Country) DETACH DELETE n")
client.run("MATCH (n:Region) DETACH DELETE n")
client.run("MATCH (n:Persons) DETACH DELETE n")
client.run("MATCH (n:ProductSearch) DETACH DELETE n")
client.run("MATCH (n:Purchase) DETACH DELETE n")
client.run("MATCH (n:ProductReservation) DETACH DELETE n")

#### 2. Load cities

In [129]:
cities = list(EntityLoader.load_filter_map('database/cities.json', CityFilter(), CityMapper()))

print(f'Cities: {len(cities)}')

Cities: 120342


In [130]:
# cityRepository.upsert(cities)

#### 3. Load countries

In [131]:
countries = list(EntityLoader.load_filter_map('database/countries.json', CountryFilter(), CountryMapper()))

print(f'Countries: {len(countries)}')

Countries: 199


In [132]:
countryRepository.upsert(countries)

#### 4. Load global regions

In [133]:
regions = list(EntityLoader.load_map('database/regions.json', RegionMapper()))

print(f'Global Regions: {len(regions)}')

Global Regions: 7


In [134]:
regionRepository.upsert(regions)

#### 5. Load users

In [135]:
users = list(EntityLoader.load_map('database/users.json', UserMapper()))

print(f'Users: {len(users)} / {users[0]}')

Users: 7 / User(id=1, email='adrian.marino@almundo.com', realm='ALMUNDO-ALMUNDO-EMPLOYEE')


In [136]:
userRepository.upsert(users)

Users:

In [137]:
adrian = "adrian.marino@almundo.com"
juanma = "juanmanuel.bouvier@almundo.com"
sofi   = "sofia.massobrio@almundo.com"
facu   = "facundo.etchanchu@almundo.com"
kevo   = "kevin.klaus@almundo.com"
gonza  = "gonzalo.prieto@almundo.com"
diego  = "diego.saez@almundo.com"

#### 6. Add citiy belong to country relation

In [138]:
def city_belong_to_country(relation):
    client.run("""
        MATCH
            (city: City {iata_code: "%s"}),
            (country: Country {code: "%s"})
        WITH
            city,
            country
        MERGE
            (city)-[:BELONG_TO]->(country)
    """ %(relation['city'],relation['country'])
)

In [139]:
relation = list(EntityLoader.load('database/city-country.json'))
print(f'Relations: {len(relation)}')

Relations: 7


In [140]:
for r in relation: city_belong_to_country(r)

#### 7. Add country belong to region relation

In [141]:
def country_belong_to_region(relation):
    client.run("""
        MATCH
            (country: Country {code: "%s"}),
            (region: Region {code: "%s"})
        WITH
            country,
            region
        MERGE
            (country)-[:BELONG_TO]->(region)
    """ %(relation['country'],relation['region'])
)

In [142]:
relations = list(EntityLoader.load('database/country-region.json'))
print(f'Relations: {len(relations)}')

Relations: 8


In [143]:
for r in relations: country_belong_to_region(r)

#### 8. Create products searches

In [147]:
def flight_search(
    email,
    from_city,
    to_city,
    from_datetime="2021-01-01T20:00:0",
    to_datetime="2021-01-15T20:00:0",
    adults=2,
    children=2,
    infants=1,
    purchase_datetime="2020-08-16T20:00:0"
):
    client.run("""
        MATCH
            (fromCity: City {iata_code: "%s"}), 
            (toCity: City {iata_code: "%s"}),
            (user: User {email: "%s"})
        WITH
            fromCity, toCity, user
        CREATE
            (search:ProductSearch {
                product_type: 'FLIGHT',
                from: datetime("%s"),
                to: datetime("%s")
            }),
            (search)-[:FROM_LOCATION]->(fromCity), 
            (search)-[:TO_LOCATION]->(toCity),
            (search)-[:FOR]->(:Persons{adults:%s, children: %s, infants: %s}),
            (user)-[:SEARCHED {datetime: datetime("%s")}]->(search)
        
        """ % (from_city, to_city, email, from_datetime, to_datetime, adults, children, infants, purchase_datetime)
    )

In [148]:
def hotel_search(
    email,
    city,
    from_datetime="2021-01-01T20:00:0",
    to_datetime="2021-01-15T20:00:0",
    adults=2,
    children=2,
    infants=1,
    purchase_datetime="2020-08-16T20:00:0"
):
    client.run("""
        MATCH
            (city: City {iata_code: "%s"}),
            (user: User {email: "%s"})
        WITH
            city, user
        CREATE
            (search:ProductSearch {
                product_type: 'HOTEL',
                from: datetime("%s"),
                to: datetime("%s")
            }),
            (search)-[:IN_LOCATION]->(city), 
            (search)-[:FOR]->(:Persons{adults: %s, children: %s, infants: %s}),
            (user)-[:SEARCHED {datetime: datetime("%s")}]->(search)
        
        """ % (city, email, from_datetime, to_datetime, adults, children, infants, purchase_datetime)
    )

Adrian's searches: 

In [150]:
flight_search(email=adrian, from_city="BUE", to_city="MIA")
flight_search(email=adrian, from_city="BUE", to_city="BCN")
flight_search(email=adrian, from_city="BUE", to_city="SLA")

hotel_search(email=adrian, city="MIA")
hotel_search(email=adrian, city="BCN")

Juan manuel searches:

In [151]:
flight_search(email=juanma, from_city="COR", to_city="SLA")

Sofia searches:

In [152]:
flight_search(email=sofi, from_city="BUE", to_city="RIO")
flight_search(email=sofi, from_city="RIO", to_city="BCN")
flight_search(email=sofi, from_city="BUE", to_city="MIA")
flight_search(email=sofi, from_city="COR", to_city="SLA")
flight_search(email=sofi, from_city="SLA", to_city="COR")

Kevin searches:

In [153]:
flight_search(email=kevo, from_city="COR", to_city="SLA")
flight_search(email=kevo, from_city="BUE", to_city="MIA")

hotel_search(email=kevo, city="MIA")
hotel_search(email=kevo, city="COR")
hotel_search(email=kevo, city="SLA")

Facundo searches:

In [154]:
hotel_search(email=facu, city="BCN")
hotel_search(email=facu, city="MIA")
hotel_search(email=facu, city="RIO")

flight_search(email=facu, from_city="BUE", to_city="RIO")
flight_search(email=facu, from_city="BUE", to_city="SLA")
flight_search(email=facu, from_city="BUE", to_city="BCN")

Diego searches:

In [155]:
hotel_search(email=diego, city="BCN")
hotel_search(email=diego, city="MIA")
hotel_search(email=diego, city="RIO")

flight_search(email=diego, from_city="BUE", to_city="RIO")
flight_search(email=diego, from_city="BUE", to_city="SLA")
flight_search(email=diego, from_city="BUE", to_city="BCN")

#### 9. Add product purchases

In [156]:
def flight_purchease(
    email,
    from_city,
    to_city,
    from_datetime="2021-01-01T20:00:0",
    to_datetime="2021-01-15T20:00:0",
    adults=2,
    children=2,
    infants=1,
    purchase_datetime="2020-08-16T20:00:0"
):
    client.run("""
        MATCH 
            (fromCity: City {iata_code: "%s"}),
            (toCity: City {iata_code: "%s"}),
            (user: User{email: "%s"})
        
        WITH
            fromCity, toCity, user
        
        CREATE
            (reservation: ProductReservation {
                product_type: 'FLIGHT',
                from: datetime("%s"),
                to: datetime("%s")
            }),
            
            (reservation)-[:FROM_LOCATION]->(fromCity),
            (reservation)-[:TO_LOCATION]->(toCity),

            (reservation)-[:FOR]->(:Persons{adults: %s, children: %s, infants: %s}),
            
            (user)-[:MAKE_A]->(:Purchase {datetime: datetime("%s")})-[:CONTAINS]->(reservation)

        """ % (from_city, to_city, email, from_datetime, to_datetime, adults, children, infants, purchase_datetime)
    )

In [159]:
def hotel_purchease(
    email,
    city,
    from_datetime="2021-01-01T20:00:0",
    to_datetime="2021-01-15T20:00:0",
    adults=2,
    children=2,
    infants=1,
    purchase_datetime="2020-08-16T20:00:0"
):
    client.run("""
        MATCH 
            (city: City {iata_code: "%s"}),
            (user: User{email: "%s"})
        WITH
            city, user
        CREATE
            (reservation: ProductReservation {
                product_type: 'HOTEL',
                from: datetime("%s"),
                to: datetime("%s")
            }),
            (reservation)-[:IN_LOCATION]->(city),
            (reservation)-[:FOR]->(:Persons{adults: %s, children: %s, infants: %s}),
            (user)-[:MAKE_A]->(:Purchase {datetime: datetime("%s")})-[:CONTAINS]->(reservation)
 
        """ % (city, email, from_datetime, to_datetime, adults, children, infants, purchase_datetime)
    )

Adrian purchases:

In [160]:
flight_purchease(email=adrian, from_city="BUE", to_city="MIA")
hotel_purchease(email=adrian, city="MIA")

flight_purchease(email=adrian, from_city="BUE", to_city="RIO")
hotel_purchease(email=adrian, city="RIO")

flight_purchease(email=adrian, from_city="COR", to_city="SLA")
hotel_purchease(email=adrian, city="SLA")

Juan manuel purchases:

In [ ]:
flight_purchease(email=juanma, from_city="COR", to_city="SLA")
hotel_purchease(email=juanma, city="SLA")

flight_purchease(email=juanma, from_city="SLA", to_city="RIO")
hotel_purchease(email=juanma, city="RIO")

flight_purchease(email=juanma, from_city="SLA", to_city="RIO")
hotel_purchease(email=juanma, city="RIO")

Sofia purchases:

In [ ]:
flight_purchease(email=sofi, from_city="BUE", to_city="RIO")
hotel_purchease(email=sofio, city="RIO")

flight_purchease(email=sofi, from_city="BUE", to_city="SLA")
hotel_purchease(email=sofio, city="SLA")

Kevin purchases:

In [ ]:
hotel_purchease(email=kevo, city="BCN")

flight_purchease(email=kevo, from_city="COR", to_city="SLA")
hotel_purchease(email=kevo, city="SLA")

Facundo purchases:

In [ ]:
hotel_purchease(email=facu, city="MIA")
hotel_purchease(email=facu, city="BUE")
hotel_purchease(email=facu, city="BCN")
hotel_purchease(email=facu, city="RIO")
hotel_purchease(email=facu, city="COR")
hotel_purchease(email=facu, city="SLA")

Gonza purchases:

In [ ]:
flight_purchease(email=gonza, from_city="BUE", to_city="MIA")
hotel_purchease(email=gonza, city="MIA")

flight_purchease(email=gonza, from_city="BUE", to_city="RIO")
hotel_purchease(email=gonza, city="RIO")

flight_purchease(email=gonza, from_city="BUE", to_city="BCN")
hotel_purchease(email=gonza, city="BCN")

## Query trends and recomendations

#### 1. Give me top 3 more purchased products in the last 10 days by product type and destination scored by purchases count.

In [ ]:
QUERY1 = """
MATCH   (:User)-[:MAKE_A]->
        (p:Purchase)-[:CONTAINS]->
        (r:ProductReservation)-[:IN_LOCATION|:TO_LOCATION]->
        (c:City)
WHERE
    p.datetime >= (datetime() - duration('P10D'))
RETURN
    r.product_type as product_type,
    c.name as destination,
    COUNT(*) as purchases
ORDER BY
    purchases DESC, 
    product_type ASC, 
    destination ASC
LIMIT 3
"""
columns=['product_type', 'destination', 'purchases']

client.query(lambda tx: to_table(tx.run(QUERY1), columns))

#### 2. Same to 1. but top 3 more searched.

In [ ]:
QUERY2 = """
MATCH   (:User)-[s:SEARCHED]->
        (ps:ProductSearch)-[:IN_LOCATION|:TO_LOCATION]->
        (c:City)
WHERE
    s.datetime >= (datetime() - duration('P10D'))
RETURN
    ps.product_type as product_type, 
    c.name as destination,
    COUNT(*) as searches
ORDER BY
    searches DESC,
    product_type ASC,
    destination ASC
LIMIT 3
"""
columns=['product_type', 'destination', 'searches']

client.query(lambda tx: to_table(tx.run(QUERY2), columns))

#### 3. For an user who bought a product give me products purchased by other users that share same destination. As result has a table with product type, destination, and a score that is the destinations count by product type ordered descendent. 

In [ ]:
def products_recomendations_by_destination_for_purchaser_user(email, time_window):
    return """
        MATCH
            (u:User {email: "%s"})
                -[:MAKE_A]->(up:Purchase)
                -[:CONTAINS]->(u_pr:ProductReservation)
                -[:TO_LOCATION|:IN_LOCATION]->(u_city:City),
            (ou:User)
                -[:MAKE_A]->(oup:Purchase)
                -[:CONTAINS]->(ou_pr:ProductReservation)
                -[:TO_LOCATION|:IN_LOCATION]->(ou_city:City)
        WHERE
            ou <> u
            AND ou_city <> u_city 
            AND up.datetime >= (datetime() - duration('P%sD')) 
            AND oup.datetime >= (datetime() - duration('P%sD'))
        WITH
            distinct(ou_pr) as product
        MATCH
            (product)-[:TO_LOCATION|:IN_LOCATION]->(city:City)
        WITH
            product.product_type as product_type,
            city.name as destination,
            REDUCE (sc=0, c in COLLECT(city) | sc+1 ) AS score
        RETURN
            product_type as product,
            destination,
            score
        ORDER BY
            score DESC
        LIMIT 10
    """ % (email, time_window, time_window)

In [ ]:
query = products_recomendations_by_destination_for_purchaser_user(
    email="adrian.marino@almundo.com",
    time_window = 60  # In days
)
columns_to_show = ['product', 'destination', 'score']

client.query(lambda tx: to_table(tx.run(query), columns_to_show))

#### 4. For an user who search given products give me products purchased by other users that share same destination. As result has a table with product type, destination, and a score that is the destinations count by product type ordered descendent. 

In [ ]:
def products_recomendations_by_destination_for_searcher_user(email, time_window):
    return """
        MATCH
            (u:User {email: "%s"})
                -[:SEARCHED]->(up:ProductSearch)
                -[:TO_LOCATION|:IN_LOCATION]->(u_city:City),
            (ou:User)
                -[:MAKE_A]->(oup:Purchase)
                -[:CONTAINS]->(ou_pr:ProductReservation)
                -[:TO_LOCATION|:IN_LOCATION]->(ou_city:City)
        WHERE
            ou <> u
            AND ou_city <> u_city 
            AND up.datetime >= (datetime() - duration('P%sD')) 
            AND oup.datetime >= (datetime() - duration('P%sD'))
        WITH
            distinct(ou_pr) as product
        MATCH
            (product)-[:TO_LOCATION|:IN_LOCATION]->(city:City)
        WITH
            product.product_type as product_type,
            city.name as destination,
            REDUCE (sc=0, c in COLLECT(city) | sc+1 ) AS score
        RETURN
            product_type as product,
            destination,
            score
        ORDER BY
            score DESC
        LIMIT 10
    """ % (email, time_window, time_window)

In [ ]:
query = products_recomendations_by_destination_for_searcher_user(
    email="juanmanuel.bouvier@almundo.com",
    time_window = 60  # In days
)
columns_to_show = ['product', 'destination', 'score']

client.query(lambda tx: to_table(tx.run(query), columns_to_show))

5. 4. Agregarle destinos que comparten un mismo pais o region pero a estos darle un score menor. Es decir los prodcutos que comparten el destino